In [6]:
import json
# with open('./DAIL-SQL/dataset/process/SPIDER-TEST_SQL_0-SHOT_CTX-200_ANS-2048/questions.json' , 'r') as f:
#     generated_prompts_file_byte = f.read()
#     generated_prompts = json.loads(generated_prompts_file_byte)

with open('./llama_pred/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048_evidence_Llama_7b.json' , 'r') as f:
    generated_response_file_byte = f.read()
    generated_response = json.loads(generated_response_file_byte)

In [7]:
with open('./llama_pred/BIRD-TEST_SQL_0-SHOT_CTX-200_ANS-2048_evidence_Llama_7b.txt' , 'w')as f:
    for i in generated_response['questions']:
        f.write( i['response']+'\n' )

In [1]:
import pickle
with open('output_sequences.pkl', 'rb') as f:  # open a text file
    output_sequences = pickle.load(f)

In [2]:
import json
import time
def extract_prompt_from_list_of_questions(question_list):
    batch_list = []
    for i in question_list:
        batch_list.append(i['prompt'])
    return batch_list
    
def generate_answer_from_LLM(prompt_list , pipeline):
    sequences = pipeline(
        prompt_list,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        # eos_token_id=tokenizer.convert_tokens_to_ids(';'),
        max_new_tokens=200,
    )
    return sequences

from utils.post_process import get_exec_output

# import asyncio
def query_to_db(query , db_id , db_dir):
    db_path = f"{db_dir}/{db_id}/{db_id}"
    flag, denotation = get_exec_output(
            db_path,
            query)
    return flag, denotation

from utils.post_process import result_eq
def put_responses_back_to_json_dataset(index , json_dataset , sequences, dataset_type='spider'):#dataset_type=spider/bird
    db_dir = './DAIL-SQL/dataset/'+ dataset_type +'/database'
    execution_accuracy = 0
    for i , seq in enumerate(sequences):
        # print(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
        # if i%10==0:
        with open('./log.txt', 'a') as f:
            f.write(f"Number of processed sequences: {i}\t|\tNumber of correct queries: {execution_accuracy} \n")
        prompt_len = len ( json_dataset['questions'][index+i]['prompt'] )
        qustion = json_dataset['questions'][index+i]['prompt'].splitlines()[-2]
        start_of_answer = json_dataset['questions'][index+i]['prompt'].splitlines()[-1]
        ground_truth = start_of_answer + ' ' + json_dataset['questions'][index+i]['response']
        gen_text = seq[0]['generated_text'][prompt_len:]
        processed_gen_text = post_process_get_sql_from_gentext(gen_text)
        # print('Question: ' , qustion )
        # print('GroundTruth answer: ' ,  ground_truth )
        # print('Generated answer: ' , gen_text )
        # print('processed_gen_text: ' , processed_gen_text )
        db_id = json_dataset['questions'][index+i]['db_id']
        with open('./log.txt', 'a') as f:
            f.write(f"proccessing the processed_gen_text:\n{processed_gen_text}\n")
        flag1, denotation1 = query_to_db(processed_gen_text , db_id , db_dir) #flag has ('result' , [data in columns])
        with open('./log.txt', 'a') as f:
            f.write(f"proccessing the ground_truth:\n{ground_truth}\n")
        flag2, denotation2 = query_to_db(ground_truth , db_id , db_dir)
        if flag2[0] != 'result':
            print('The following ground truth has an error:' , ground_truth)
            is_equal = False
        elif flag1[0] != 'result':
            is_equal = False
            # print(flag1[0] , ' --> ' , 'processed_gen_text: ' , processed_gen_text )
        elif 'ORDER BY' in ground_truth or 'order by' in ground_truth:
            is_equal = result_eq(flag1[1] , flag2[1] , order_matters=True)
        else:
            is_equal = result_eq(flag1[1] , flag2[1] , order_matters=False)
        json_dataset['questions'][index+i]['response'] = processed_gen_text
        execution_accuracy += is_equal
        # print('is_equal: ', is_equal)
        # print('--------------------------')
    return execution_accuracy

from utils.post_process import process_duplication
def post_process_get_sql_from_gentext(gen_text):
    # remove \n and extra spaces
    sql = " ".join(gen_text.replace("\n", " ").split())
    sql = process_duplication(sql)
    # python version should >= 3.8
    if sql.startswith("SELECT"):
        sql = sql
    elif sql.startswith(" "):
        sql = "SELECT" + sql
    else:
        sql = "SELECT " + sql
    return sql

def process_a_batch(question_units, pipeline, pipeline_sequences ,pipeline_index):
    start_time = time.time()
    batch_list = extract_prompt_from_list_of_questions(question_units)
    sequences = generate_answer_from_LLM(batch_list , pipeline)
    pipeline_sequences.extend(sequences)
    # print(f"{pipeline_index}: batch_time: ", time.time() - start_time )
    return pipeline_sequences
    
def pipline_process(pipeline , data , batch_size, pipeline_index):
    start_time = time.time()
    pipeline_sequences = []
    chunk_size = len(data)
    number_of_batches = int(chunk_size/batch_size)
    for index in range( number_of_batches ):
        question_units = data[ index*batch_size : index*batch_size+batch_size ]
        pipeline_sequences = process_a_batch(question_units, pipeline, pipeline_sequences ,pipeline_index)
        if ( index*batch_size + batch_size < chunk_size ) and ( index*batch_size + 2*batch_size > chunk_size ):
            question_units = data[index*batch_size + batch_size :]
            pipeline_sequences = process_a_batch(question_units, pipeline, pipeline_sequences, pipeline_index)
    with open('./log.txt', 'a') as f:
        f.write(f"{pipeline_index}: pipeline_process_time: {time.time() - start_time} \n")
    # print('pipeline_process_time: ', time.time() - start_time )
    return pipeline_sequences
    


In [3]:
import json
with open('./DAIL-SQL/dataset/process/BIRD-TEST_SQL_5-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048/questions.json' , 'r') as f:
    generated_prompts_file_byte = f.read()
    generated_prompts = json.loads(generated_prompts_file_byte)

data_size = len(generated_prompts['questions'])

In [4]:
exec_acc = put_responses_back_to_json_dataset(0 , generated_prompts , output_sequences , dataset_type='bird')

print('execution accuracy = ' , exec_acc/data_size)
with open('./log.txt', 'a') as f:
    f.write(f"execution accuracy = {exec_acc/data_size} \n")
with open('./llama_pred/BIRD-TEST_SQL_5-SHOT_EUCDISMASKPRESKLSIMTHR_QA-EXAMPLE_CTX-200_ANS-2048_Llama_2_7b.json' , 'w' )as f:
    json.dump(generated_prompts , f)

The following ground truth has an error: SELECT  bQuery AS (SELECT DISTINCT T1.atom_id, T1.element, T1.molecule_id, T2.label FROM atom AS T1 INNER JOIN molecule AS T2 ON T1.molecule_id = T2.molecule_id WHERE T2.molecule_id = 'TR006') SELECT CAST(COUNT(CASE WHEN element = 'h' THEN atom_id ELSE NULL END) AS REAL) / (CASE WHEN COUNT(atom_id) = 0 THEN NULL ELSE COUNT(atom_id) END) AS ratio, label FROM SubQuery GROUP BY label
The following ground truth has an error: SELECT  xBanned AS (SELECT format, COUNT(*) AS count_banned FROM legalities WHERE status = 'Banned' GROUP BY format ORDER BY COUNT(*) DESC LIMIT 1) SELECT T2.format, T1.name FROM cards AS T1 INNER JOIN legalities AS T2 ON T2.uuid = T1.uuid INNER JOIN MaxBanned MB ON MB.format = T2.format WHERE T2.status = 'Banned'
The following ground truth has an error: SELECT  me_in_seconds AS ( SELECT T1.positionOrder, CASE WHEN T1.positionOrder = 1 THEN (CAST(SUBSTR(T1.time, 1, 1) AS REAL) * 3600) + (CAST(SUBSTR(T1.time, 3, 2) AS REAL) * 60)

In [6]:
#574
#885
#1238
prompt_len = len ( generated_prompts['questions'][1238]['prompt'] )
output_sequences[1238][0]['generated_text'] = output_sequences[1238][0]['generated_text'][prompt_len:]

In [8]:
print('hello world')

hello world


In [2]:
from utils.post_process import get_exec_output
db_dir = './DAIL-SQL/dataset/spider/database'
# import asyncio
def query_to_db(query , db_id):
    db_path = f"{db_dir}/{db_id}/{db_id}"
    flag, denotation = get_exec_output(
            db_path,
            query)
    return flag, denotation
    

from utils.post_process import result_eq
execution_accuracy = 0
counter = 0
for q_unit_gen , q_unit_truth in zip(generated_response['questions'] , generated_prompts['questions']):
    pred_response = q_unit_gen['response']
    start_of_answer = q_unit_truth['prompt'].splitlines()[-1]
    ground_truth = start_of_answer + ' ' + q_unit_truth['response']
    db_id = q_unit_truth['db_id']
    flag1, denotation1 = query_to_db(pred_response , db_id) #flag has ('result' , [data in columns])
    flag2, denotation2 = query_to_db(ground_truth , db_id)
    if flag1[0] != 'result':
        is_equal = False
        # print(  counter , '-' , flag1[0] , ' --> ' , 'pred_response: ' , pred_response )
    elif 'ORDER BY' in ground_truth or 'order by' in ground_truth:
        is_equal = result_eq(flag1[1] , flag2[1] , order_matters=True)
    else:
        is_equal = result_eq(flag1[1] , flag2[1] , order_matters=False)
    execution_accuracy += is_equal
    counter += 1
print( execution_accuracy/len(generated_response['questions']) )

0.9497098646034816
